In [4]:
import numpy as np
import math

In [2]:
# For forward propagation
# Requirements - 
# g(function)

In [3]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [4]:
a = np.matrix([[1],[2],[3]])

In [5]:
a


matrix([[1],
        [2],
        [3]])

In [6]:
b = a.getT().getA()[0]

In [7]:
softmax(b)

array([ 0.09003057,  0.24472847,  0.66524096])

In [8]:
softmax(a)

matrix([[ 0.09003057],
        [ 0.24472847],
        [ 0.66524096]])

In [9]:
b.tolist()[0]

1

In [5]:
class neuralNetwork:
    numberOfLayers = 0 #Input neurons are counted as one layer
    lengthOfFirstLayer = 0 #Including bias
    lengthOfLastLayer = 0 #Including bias
    learningRate = 0.1
    activationVals = {}
    thetaVals = {}
    delVals = {}
    
    def __init__(self,numberOfLayers, lengthOfFirstLayer, lengthOfLastLayer):
        self.numberOfLayers = numberOfLayers
        self.lengthOfFirstLayer = lengthOfFirstLayer
        self.lengthOfLastLayer = lengthOfLastLayer
        self.learningRate = 0.1
        self.initializeTheta()
        self.initializeActivation()

    def initializeTheta(self): #Should put random numbers between -1 and 1
        for i in range(1,self.numberOfLayers-1):
            self.thetaVals[i] = np.matrix(np.random.randn(self.lengthOfFirstLayer, self.lengthOfFirstLayer))
        self.thetaVals[self.numberOfLayers-1] = np.matrix(np.random.randn(self.lengthOfLastLayer, self.lengthOfFirstLayer))

    def initializeActivation(self):
        for i in range(1,self.numberOfLayers):
            self.activationVals[i] = np.matrix(np.zeros((self.lengthOfFirstLayer,1)))
        self.activationVals[self.numberOfLayers] = np.matrix(np.zeros((self.lengthOfLastLayer,1)))
    
    def g(self, product):
        result = 1/(1+math.exp(-product))
        return result
        
    def printVals(self):
        print self.numberOfLayers, self.lengthOfFirstLayer, self.lengthOfLastLayer
        print self.thetaVals
        print self.activationVals
    
    #Input is a matrix of single column
    def softmax(self,x):
        assert(x.shape[1] == 1)
        returnVal = x.getT().getA()[0] #Getting the array
        returnVal = np.exp(returnVal) / np.sum(np.exp(returnVal), axis=0)
        returnVal = np.matrix(returnVal).getT()
        return returnVal
    
    # Mostly working
    # Can use vectorize - came to know about it later
    def forwardPropagation(self, features): #Features includes no bias
        assert(features.shape[1]==1)
        self.activationVals[1] = np.append(np.zeros((1,1)),features,axis=0) #Added bias
        self.activationVals[1][0] = 1
        assert(self.activationVals[1].shape[0] == features.shape[0] + 1)
        assert(self.activationVals[1].shape[1] == 1)
        
        for layer in range(2, self.numberOfLayers + 1):
            print layer
            self.activationVals[layer] = self.thetaVals[layer-1] * self.activationVals[layer-1]
            self.activationVals[layer][0] = 1
            numberOfNeurons = self.lengthOfFirstLayer
            if layer == self.numberOfLayers:
                numberOfNeurons = self.lengthOfLastLayer
            for neuronNumber in range(1, numberOfNeurons):
                self.activationVals[layer][neuronNumber] = self.g(self.activationVals[layer][neuronNumber])
        
        return self.softmax(self.activationVals[self.numberOfLayers][1:,0])
    
    #Ran forwardProp algo separately before
    def backPropagation(self, expectedResult):
        expectedResultWithBias = np.append(np.zeros((1,1)),expectedResult,axis=0)
        expectedResultWithBias[0] = 1
        resultGot = self.activationVals[self.numberOfLayers]
        assert(expectedResultWithBias.shape == resultGot.shape)
        self.delVals[self.numberOfLayers] = self.activationVals[self.numberOfLayers] - expectedResultWithBias
        assert(self.delVals[self.numberOfLayers][0] == 0)
        for layer in list(reversed(range(2,self.numberOfLayers))):
            derivativeTerm = np.multiply(self.activationVals[layer], 1 - self.activationVals[layer])
            nonDerivativeTerm = self.thetaVals[layer].getT()*self.delVals[layer+1]
            self.delVals[layer] = np.multiply(nonDerivativeTerm, derivativeTerm)
            assert(self.delVals[layer][0] == 0)
        
    def changeTheta(self):
        for layer in range(1,self.numberOfLayers):
            DelMatrix = self.delVals[layer+1]*self.activationVals[layer].getT()
            self.thetaVals[layer] = self.thetaVals[layer] - self.learningRate*DelMatrix
        
        """
        for layer in range(1,self.numberOfLayers-1): #If 3 layers, we have to do only 1
            for i in range(self.lengthOfFirstLayer):
                for j in range(self.lengthOfFirstLayer):
                    self.thetaVals[layer][i,j] = self.thetaVals[layer][i,j] - \
                                                    self.learningRate*self.activationVals[layer][j]*self.delVals[layer+1][i]
        layer = self.numberOfLayers-1
        for i in range(self.lengthOfLastLayer):
            for j in range(self.lengthOfFirstLayer):
                self.thetaVals[layer][i,j] = self.thetaVals[layer][i,j] - \
                                                self.learningRate*self.activationVals[layer][j]*self.delVals[layer+1][i]
        
        # Internal layers
        for layer in range(2,self.numberOfLayers):
            for neuronNumber in range(self.lengthOfFirstLayer+1):
                self.activationVals[layer][neuronNumber] = g(self.thetaVals[layer-1][neuronNumber,:]*self.activationVals[layer-1])
            self.activationVals[layer][0] = 1
        
        #Outer layer
        for neuronNumber in range(self.lengthOfLastLayer+1):
            self.activationVals[self.numberOfLayers][neuronNumber] = g(self.thetaVals[self.numberOfLayers-1][neuronNumber,:]*
                                                                      self.activationVals[self.numberOfLayers-1])
        return self.activationVals[self.numberOfLayers][1:,0]
        """

In [96]:
#Testing the neural network

In [6]:
a = neuralNetwork(3,4,3)
features = np.matrix([1,1,1]).getT()

In [7]:
a.forwardPropagation(features)
b = np.matrix([[1],[0]])
a.backPropagation(b)
a.changeTheta()
a.thetaVals

2
3


{1: matrix([[ 0.46296093, -0.0651621 ,  1.09753935,  0.67522809],
         [-0.6969412 ,  0.34752842,  0.79617536, -2.91191235],
         [-2.40267916,  0.10302967,  0.40279592, -0.6320986 ],
         [ 0.26451742,  0.99752832, -0.53873337, -1.67936602]]),
 2: matrix([[ 1.08481981, -0.09323942,  2.33786436, -0.31965006],
         [ 0.0652104 ,  0.6850355 , -0.88040248, -1.0650807 ],
         [-1.00440485,  2.49838085,  2.0856692 ,  0.14770949]])}

In [8]:
a.forwardPropagation(features)
b = np.matrix([[1],[0]])
a.backPropagation(b)
a.changeTheta()
a.thetaVals

2
3


{1: matrix([[ 0.46296093, -0.0651621 ,  1.09753935,  0.67522809],
         [-0.70050621,  0.34396341,  0.79261035, -2.91547737],
         [-2.41106303,  0.0946458 ,  0.39441204, -0.64048247],
         [ 0.25151023,  0.98452112, -0.55174057, -1.69237321]]),
 2: matrix([[ 1.08481981, -0.09323942,  2.33786436, -0.31965006],
         [ 0.12122824,  0.68942381, -0.87626538, -1.04952628],
         [-1.03952779,  2.4956294 ,  2.08307525,  0.13795693]])}

In [9]:
a.forwardPropagation(features)
b = np.matrix([[1],[0]])
a.backPropagation(b)
a.changeTheta()
a.thetaVals

2
3


{1: matrix([[ 0.46296093, -0.0651621 ,  1.09753935,  0.67522809],
         [-0.70390403,  0.34056559,  0.78921253, -2.91887518],
         [-2.4189363 ,  0.08677253,  0.38653877, -0.64835574],
         [ 0.23944545,  0.97245634, -0.56380534, -1.70443799]]),
 2: matrix([[ 1.08481981, -0.09323942,  2.33786436, -0.31965006],
         [ 0.17543824,  0.69361501, -0.87238438, -1.03503296],
         [-1.07358907,  2.49299598,  2.08063674,  0.12885047]])}

In [10]:
a.forwardPropagation(features)
b = np.matrix([[1],[0]])
a.backPropagation(b)
a.changeTheta()
a.thetaVals

2
3


{1: matrix([[ 0.46296093, -0.0651621 ,  1.09753935,  0.67522809],
         [-0.7071497 ,  0.33731992,  0.78596686, -2.92212086],
         [-2.42634849,  0.07936034,  0.37912659, -0.65576793],
         [ 0.228231  ,  0.96124189, -0.5750198 , -1.71565244]]),
 2: matrix([[ 1.08481981, -0.09323942,  2.33786436, -0.31965006],
         [ 0.22791885,  0.69762193, -0.86873557, -1.02149247],
         [-1.10665192,  2.49047162,  2.07833798,  0.12031995]])}

In [102]:
a.backPropagation(b)

In [103]:
a.changeTheta()

In [104]:
a.thetaVals

{1: matrix([[ 1.25442967,  1.05507052, -0.19241158,  0.45980842],
         [ 1.34737066, -2.75135218, -0.3032265 ,  0.06879914],
         [ 1.04687633,  1.22935677, -0.85185639, -0.38457311],
         [ 0.67571302, -0.20956889,  1.14300245, -0.20112442]]),
 2: matrix([[ 2.35215101, -0.3904102 , -1.02787376,  0.75406666],
         [ 0.0632558 ,  1.30607959,  1.03671232,  0.59822863],
         [ 0.77317763, -0.84469085, -0.3308197 ,  0.52432102]])}